In [1]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 81.1/81.1 kB 1.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=bd87121bc981e7661ac8540e7cef6f57e877799230437bfb5456481984fdac9a
  Stored in directory: c:\users\kiit\appdata\local\pip\cache\wheels\65\7a\a7\78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k


In [2]:
from langchain_community.tools import WikipediaQueryRun

## wrapper class
from langchain_community.utilities import WikipediaAPIWrapper



In [14]:
## for wikkipedia api langchain has its api so no need for providing api

api_wrapper =  WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki =  WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
wiki.name

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

vectrodb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectrodb.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000013F4C20CFD0>)

In [11]:
from langchain.tools.retriever import create_retriever_tool

## creating a tool to search retrieval for that page
retriever_tool = create_retriever_tool(retriever, "langsmith_search", 
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [12]:
retriever_tool.name

'langsmith_search'

In [13]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [16]:
tools = [wiki, arxiv, retriever_tool]

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Projects\\Langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000013F3C5644C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstor

In [20]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [22]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [24]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent= agent, tools = tools, verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [25]:
agent_executer.invoke({"input": "Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We prov

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. LangSmith works on its own and does not require the use of LangChain. \n\nTo get started with LangSmith, you can follow these steps:\n1. Install LangSmith using Python or TypeScript.\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n3. Set up your environment by exporting necessary variables like LANGCHAIN_TRACING_V2 and LANGCHAIN_API_KEY.\n4. Log your first trace and evaluate your application using LangSmith.\n\nFor more detailed information and tutorials on using LangSmith, you can visit their official website at [LangSmith](https://langsmith.com/).'}

In [26]:
agent_executer.invoke({"input": "Tell me about machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Machine learning`


